In [ ]:
colors = [
    "#4b2e83",  # Purple
    "#b7a57a",  # Gold
    "#d9d9d6",  # Light Gray
    "#5e6a71",  # Dark Gray
    "#000000",  # Black
    "#ffffff",  # White
    "#85754d",  # Bright Gold
    "#cfcfcd"   # Cool Gray
]

# Load libraries

In [ ]:
import pandas as pd
import os
import ast
import time
import numpy as np
import sys
import json
from IPython.display import clear_output
import asyncio

In [ ]:
from gql import gql
import json
import pandas as pd
import sys
import warnings
from sqlalchemy import text
import re
from tqdm import tqdm
import requests

from system_prompt import get_prompt

sys.path.append('../src') 
from connect_psql import connect_psql
from connect_anthropic import connect_anthropic


In [ ]:
import anthropic
import logging

In [ ]:
tqdm.pandas()

## Load data

In [ ]:
df = pd.read_csv('../input/sample.csv')
df['created_date'] = pd.to_datetime(df['created'], unit='ms', errors='coerce')

In [ ]:
# remove AI thinking process from AI response messages. only label the displayed responses
df['content'] = df['content'].apply(
        lambda x: re.search(r"<output-cai>(.*?)</output-cai>", x, re.DOTALL).group(1).strip() if re.search(r"<output-cai>(.*?)</output-cai>", x, re.DOTALL) else x
    )

### ANTHROPIC


In [ ]:
claude_client = connect_anthropic()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
t = df.iloc[0:3]
len(t)

In [ ]:
batch_requests = []
for index, row in t.iterrows():
    request = row['content']
    custom_id = row['id']

    content = get_prompt(request)

    batch_requests.append({
        "custom_id": custom_id,
        "params": {
            "model": "claude-3-5-haiku-20241022",
            "max_tokens": 1200,
            "temperature": 0,
            "messages": [
                {
                    "role": "user",
                    "content": content.strip(),
                }
            ],
        },
    })

claude_client.beta.messages.batches.create(
    requests=batch_requests
)

In [ ]:
# Insert IDs from previous step output
ids = ['msgbatch_'] #edit thisline

for id in ids:
    message_batch = claude_client.beta.messages.batches.retrieve(
        id)
    print(message_batch.processing_status)

In [ ]:
# with additional headings and rationales

results_data = []

# Iterate through each result and extract its full structure
for id in ids:
    for result in claude_client.beta.messages.batches.results(id):
        # Initialize the result entry with basic details
        result_entry = {
            "custom_id": result.custom_id,
            "status": result.result.type
        }

        if result.result.type == "succeeded":
            try:
                # Extract full content as JSON from the 'text' field
                text_content = result.result.message.content[0].text.strip()
                # Ensure the text content contains JSON by looking for JSON delimiters
                json_start = text_content.find("{")
                json_end = text_content.rfind("}") + 1
                
                if json_start != -1 and json_end != 0:
                    # Extract the JSON portion
                    json_string = text_content[json_start:json_end]
                    content_data = json.loads(json_string)
                    
                    # Populate result_entry with content details and other relevant fields
                    result_entry.update({
                        "message_id": result.result.message.id,
                        "model": result.result.message.model,
                        "role": result.result.message.role,
                        "stop_reason": result.result.message.stop_reason,
                        "usage": {
                            "input_tokens": result.result.message.usage.input_tokens,
                            "output_tokens": result.result.message.usage.output_tokens
                        },
                        "content": content_data  # Insert the parsed JSON content directly
                    })
                else:
                    result_entry["error"] = text_content

            except json.JSONDecodeError as e:
                # Handle JSON parsing errors gracefully
                result_entry["error"] = text_content
            except ValueError as e:
                # Handle other content-related errors
                result_entry["error"] = str(e)

        elif result.result.type == "errored":
            # Capture error details for errored results
            result_entry["error_type"] = result.result.error.type
        elif result.result.type == "expired":
            # No additional data to add for expired status
            pass

        # Append each complete result entry to results_data
        results_data.append(result_entry)
        len(results_data)


In [ ]:
results_data[1]

In [ ]:
import pandas as pd

# Step 1: Flatten a single result entry
def flatten_result(result):
    content = result.get('content', {})
    ed_context = content.get('EdContext', {}) or {}

    base = {
        'custom_id': result.get('custom_id'),
        'message_id': result.get('message_id'),
        'model': result.get('model'),
        'status': result.get('status'),
        'role': result.get('role'),
        'stop_reason': result.get('stop_reason'),
        'input_tokens': result.get('usage', {}).get('input_tokens'),
        'output_tokens': result.get('usage', {}).get('output_tokens'),
        'ClarityAndSpecificity': content.get('ClarityAndSpecificity'),
        'Subject Area': ed_context.get('Subject Area'),
        'Grade Level': ed_context.get('Grade Level'),
        'Pedagogical Framework': ed_context.get('Pedagogical Framework')
    }

    # These are the nested domains we want to flatten
    domains = [
        'Instructional Practices',
        'Student Needs and Context',
        'Curriculum and Content Planning',
        'Assessment and Feedback',
        'Professional Responsibilities',
        'Other'
    ]

    for domain in domains:
        subdomain_dict = content.get(domain, {})
        if isinstance(subdomain_dict, dict):
            for subdomain, value in subdomain_dict.items():
                key = f"{domain} - {subdomain}"
                base[key] = value
        else:
            # if domain exists but is not a dict, skip
            continue

    return base

# Step 2: Flatten all entries
df_flat = pd.DataFrame([flatten_result(r) for r in results_data])

# Step 3: Melt to long format
id_cols = [
    'custom_id', 'message_id', 'model', 'status', 'role',
    'stop_reason', 'input_tokens', 'output_tokens',
    'ClarityAndSpecificity', 'Subject Area', 'Grade Level', 'Pedagogical Framework'
]

df_long = df_flat.melt(
    id_vars=id_cols,
    var_name='Category',
    value_name='Code'
).dropna(subset=['Code']).reset_index(drop=True)

# Split 'Category' into 'Domain' and 'Subcategory'
df_long[['domain', 'category']] = df_long['Category'].str.split(' - ', n=1, expand=True)

# Rename column and drop original Category
df_long = df_long.rename(columns={'Code': 'code_value'}).rename(columns={'custom_id': 'colleague_message_id'}).drop(columns=['Category'])


In [ ]:
df_long

In [ ]:
import pandas as pd
import json

flattened = []

for row in results_data:
    base = {
        'custom_id': row.get('custom_id'),
        'status': row.get('status'),
        'message_id': row.get('message_id'),
        'model': row.get('model'),
        'role': row.get('role'),
        'stop_reason': row.get('stop_reason'),
        'input_tokens': row.get('usage', {}).get('input_tokens'),
        'output_tokens': row.get('usage', {}).get('output_tokens')
    }

    parsed = False

    if 'content' in row and isinstance(row['content'], dict):
        flat_content = pd.json_normalize(row['content'], sep='_', max_level=10).to_dict(orient='records')[0]
        base.update(flat_content)
        flattened.append(base)
        continue

    error_text = row.get('error', '').strip()

    if error_text.startswith('{'):
        for part in error_text.split('\n\n'):
            try:
                json_obj = json.loads(part.strip())
                base_copy = base.copy()
                if 'content' in json_obj and isinstance(json_obj['content'], dict):
                    flat_content = pd.json_normalize(json_obj['content'], sep='_', max_level=10).to_dict(orient='records')[0]
                    base_copy.update(flat_content)
                else:
                    flat_content = pd.json_normalize(json_obj, sep='_', max_level=10).to_dict(orient='records')[0]
                    base_copy.update(flat_content)
                flattened.append(base_copy)
                parsed = True
            except json.JSONDecodeError:
                continue

    if not parsed:
        base['error'] = error_text
        flattened.append(base)

df_output = pd.DataFrame(flattened)


In [ ]:
df_output.columns = df_output.columns.str.replace("^content-", "", regex=True)

# Rename "custom_id" to "id"
df_output = df_output.rename(columns={"custom_id": "id"})

In [ ]:
df_output.head()

## post progress LLM results

In [ ]:
df_output.columns[12:]

#### clean up error message

In [ ]:
df_output[~df_output['error'].isna()]['error']

In [ ]:
# check if any content went to error message
mask = df_output['error'].notna() & ~(
    df_output['error'].str.contains('no message', na=False) | 
    df_output['error'].str.contains('no actual message', na=False)
)

df_filtered = df_output[mask]
df_filtered

In [ ]:
df_output.loc[mask, 'Other_Discourse Continuity'] = "Modification Request"

#### Clean up formatting and non eductional requests

In [ ]:
df_output['Other_Non-Educational'].unique()

In [ ]:
df_output['Other_Non-Educational'] = df_output['Other_Non-Educational'].apply(
    lambda x: 'Non-Educational' if x in [True, "Grove's Lawn Care"] else x
)

In [ ]:
df_output['Other_Non-Educational'] = df_output['Other_Non-Educational'].apply(
    lambda x: 'Administrative Communications' if x in ['Administrative Daily Update', 'Administrative Meeting Minutes',  'Administrative School Information', 'Graduation Speech'] else x
)

In [ ]:
df_output['Other_Non-Educational'] = df_output['Other_Non-Educational'].apply(
    lambda x: 'Modification Request' if x in ['Discourse Continuity'] else x
)

In [ ]:
df_output['Other_Discourse Continuity'].unique()

In [ ]:
df_output['Other_Discourse Continuity'] = df_output['Other_Discourse Continuity'].apply(
    lambda x: 'Modification Request' if x in ['Continue', 'Waiting for grade', 'Rewrite', 'Rewrite Instructions', 'Rewrite instructions', True, 'Continue', 'Proceed', 'Turn and Talks Request'] else x
)

#### Clean up educational labels

In [ ]:
df_post = df_output[['id',
       'Instructional Practices_Differentiation and Accessibility',
       'Instructional Practices_Explicit Teaching',
       'Instructional Practices_Project-Based and Real-World Learning',
       'Instructional Practices_Critical Thinking and Inquiry',
       'Instructional Practices_Instructional Routine',
       'Instructional Practices_Engagement and Motivation',
       'Student Needs and Context_Classroom Setting',
       'Student Needs and Context_Student Profiles',
       'Student Needs and Context_Career Readiness',
       'Curriculum and Content Planning_Planning',
       'Curriculum and Content Planning_Tech Integration',
       'Assessment and Feedback_Assessment',
       'Assessment and Feedback_Feedback',
       'Professional Responsibilities_Professional Development',
       'Professional Responsibilities_Communication', 'Other_Non-Educational',
       'Other_Discourse Continuity', 'Instructional Practices',
       'Student Needs and Context', 'Curriculum and Content Planning',
       'Professional Responsibilities', 'Other',
       'Instructional Practices_Collaborative Learning',
       'Assessment and Feedback',
       'Instructional Practices_Assessment and Feedback',
       'Instructional Practices_Assessment',
       'Instructional Practices_Tech Integration',
       'Instructional Practices_Assessment and Feedback_Assessment',
       'Instructional Practices_Feedback',
       'Curriculum and Content Planning_Assessment',
       'Student Needs and Context_Communication',
       'Instructional Practices_Planning',
       'Instructional Practices_Career Readiness', 'Non-Educational',
       'Discourse Continuity']]

In [ ]:
column_rename_map = {
    'Instructional Practices_Differentiation and Accessibility': 'Differentiated Instructional Strategies',
    'Instructional Practices_Explicit Teaching': 'Explaining Core Science Concepts',
    'Instructional Practices_Project-Based and Real-World Learning': 'Projects',
    'Instructional Practices_Critical Thinking and Inquiry': 'Encourage Critical Thinking and High-Level Cognition',
    'Instructional Practices_Instructional Routine': 'Learning Progression and Routine Adjustments',
    'Instructional Practices_Engagement and Motivation': 'Actionable Engagement Strategy',
    'Instructional Practices_Collaborative Learning': 'Group Work',
    'Instructional Practices_Tech Integration': 'Multimedia Use for Instruction',
    'Instructional Practices_Assessment': 'Generate Formative Assessments',
    
    'Student Needs and Context_Classroom Setting': 'Homeschool',
    'Student Needs and Context_Student Profiles': 'Special Education (IEP)',
    'Student Needs and Context_Career Readiness': 'Student Career Exploration',
    
    'Curriculum and Content Planning_Planning': 'Entire Lesson Planning',
    'Curriculum and Content Planning_Tech Integration': 'Multimedia Use for Instruction',
    
    'Assessment and Feedback_Assessment': 'Generate Summative Assessments',
    'Assessment and Feedback_Feedback': 'Generate Feedback to Students',
    
    'Professional Responsibilities_Professional Development': 'Professional Development Needs and Requirements',
    'Professional Responsibilities_Communication': 'Communicate with Parents or Community',
    
    'Other_Non-Educational': 'Non-Educational',
    'Other_Discourse Continuity': 'Follow-Up Prompt and Continuation',
    
    # Clean these garbage catch-alls too
    'Other': 'Non-Educational',
    'Professional Responsibilities': 'Professional Development Needs and Requirements',
    'Assessment and Feedback': 'Generate Feedback to Students',
    'Instructional Practices': 'Explaining Core Science Concepts',
    'Student Needs and Context': 'Social Emotional Support',
    'Curriculum and Content Planning': 'Entire Lesson Planning',
    'error': 'error'
}

df_output = df_output.rename(columns=column_rename_map)

In [ ]:
qual_map = pd.read_csv('../input/qual_map.csv')

In [ ]:
# Step 1: Extract all phrases per row, treating only strings
df_expanded = df_post.drop(columns='id').apply(
    lambda row: [
        phrase.strip()
        for val in row.dropna()
        if isinstance(val, str)
        for phrase in val.split(', ')
        if phrase.strip()
    ],
    axis=1
)

# Step 2: Convert to indicator matrix
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_indicators = pd.DataFrame(mlb.fit_transform(df_expanded), columns=mlb.classes_)
df_indicators.insert(0, 'id', df_post['id'].values)


In [ ]:
# Normalize function (strip spaces and lowercase)
normalize = lambda x: x.strip()

# Create normalized sets
valid_set = set(map(normalize, qual_map['updated_items']))
col_set = {col for col in df_indicators.columns if col != 'id'}

# Identify mismatches by original column names
invalid_cols = [col for col in col_set if normalize(col) not in valid_set]

pd.Series(invalid_cols).to_csv('../input/invalid_code.csv')

#### This step requires an offline review of invalid codes and manual mapping to the existing codebook. Most invalid entries resulted from variations in naming conventions, such as “special ed,” “SPED,” “IEP,” and “special ed (IEP).” Create a new column call map_label to map the org_label to appropriate labels in the codebook

#### match

In [ ]:
#upload the manually compiled map
invalid_map = pd.read_csv('../input/invalid_map.csv')

In [ ]:
for _, row in invalid_map.iterrows():
    col1 = row['org_label']
    col2 = row['map_label']
    
    if col1 in df_indicators.columns and col2 in df_indicators.columns:
        # Merge using logical OR
        df_indicators[col2] = df_indicators[[col1, col2]].max(axis=1)

In [ ]:
# Check
for _, row in invalid_map.iterrows():
    col1 = row['org_label']
    col2 = row['map_label']
    
    if col1 in df_indicators.columns and col2 in df_indicators.columns:
        merged = df_indicators[[col1, col2]].max(axis=1)
        
        # Check if the merge result matches the new col2
        if not (df_indicators[col2] == merged).all():
            print(f"Merge mismatch for: {col1} -> {col2}")